In [1]:
import numpy as np
import matplotlib.pyplot as plt
import eleanor
from astropy.timeseries import BoxLeastSquares
import pandas as pd
from scipy.signal import savgol_filter
from dave.tessPipeline.tessmastio import TessAstroqueryArchive
import dave.fileio.tpf as tpf
import requests
from astroquery.mast import Tesscut
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord

from lightkurve import LightCurve
from lightkurve import periodogram

%matplotlib inline

In [2]:
#eleanor.__version__

In [3]:
sectors_LC = [13]#[1, 3, 4, 5, 6, 7, 8, 9]#[13]#[3, 6, 10]#,3,4,5,6,7,8,9,10,11]
sectors_SC = [7,8,9]#[6, 10]#[4,5,6,7,8,9,10,11]
tic_ = 260128333#150428135#260128333#70763084#35655828#

In [4]:
localPath = "/Users/vkostov/.eleanor"
ar = TessAstroqueryArchive(localPath)
detrending_ = 'PDCSAP_FLUX'#'PDCSAP_FLUX'

planetNum = 1

In [5]:
def get_catalog_info(tic_id):
    cols = ['ID', 'ra', 'dec', 'Tmag', 'Vmag', 'Teff', 'logg', 'lumclass', 'rad', 'e_rad', 'mass', 'numcont', 'contratio']
    catalogData = Catalogs.query_object('TIC' + str(tic_id), radius = 0.002, catalog="TIC", pagesize=1).to_pandas()
    catalogData.ID = catalogData.ID.astype(int)
    catalogData = catalogData[catalogData.ID == tic_id]
    return catalogData[cols]

In [6]:
aa = get_catalog_info(tic_)

In [7]:
ra_ = aa.ra[0]
dec_ = aa.dec[0]

print(ra_, dec_)

obj_coord = SkyCoord(ra_,dec_,unit="deg")
print(obj_coord)

92.1331996217219 -59.5411333817205
<SkyCoord (ICRS): (ra, dec) in deg
    (92.13319962, -59.54113338)>


In [11]:
Tesscut.get_sectors(obj_coord)

sectorName,sector,camera,ccd
str14,int64,int64,int64
tess-s0001-4-4,1,4,4
tess-s0002-4-4,2,4,4
tess-s0003-4-1,3,4,1
tess-s0004-4-1,4,4,1
tess-s0005-4-1,5,4,1
tess-s0006-4-1,6,4,1
tess-s0007-4-2,7,4,2
tess-s0008-4-2,8,4,2
tess-s0009-4-2,9,4,2


In [9]:
cutout_hdu = Tesscut.get_cutouts(obj_coord, size=10)[-3]

In [ ]:
cutout_hdu.info()

In [ ]:
cutout_table = cutout_hdu[1].data
cutout_table.columns

In [ ]:
aa = cutout_table.FLUX
tt = cutout_table.TIME

bb = cutout_table.FLUX_ERR


In [ ]:
bb.shape

In [ ]:
bb

In [ ]:
print(aa.shape)

ff1 = np.sum(aa[:,3:6,3], axis = 1)
ff2 = np.sum(aa[:,3:6,4], axis = 1)
ff3 = np.sum(aa[:,3:6,5], axis = 1)

bb1 = np.sum(bb[:,3:6,3], axis = 1)
bb2 = np.sum(bb[:,3:6,4], axis = 1)
bb3 = np.sum(bb[:,3:6,5], axis = 1)



ff = ff1+ff2+ff3
bb = bb1+bb2+bb3

In [ ]:
plt.plot(tt, ff/np.nanmean(ff), 'k.')
#plt.plot(tt, bb, 'r.')
plt.xlim(1575, 1582)
plt.ylim(0.65, 0.7)

In [ ]:
bfig = plt.figure()
bfig.circle(weird_lightcurve["TIME"],weird_lightcurve["LC_DETREND"], fill_color="black",size=1, line_color=None)
plt.show(bfig)

In [ ]:
def get_eleanor_data(tic_, sectors_):
#    star = eleanor.Source(tic=tic_, sector=11, tc=True)
    star = eleanor.multi_sectors(tic=tic_, sectors=sectors_, tc=True)

#    plt.figure(figsize=(15,5))

    time_eleanor = np.array([])
    flux_eleanor_corr, flux_eleanor_psf = np.array([]), np.array([])
    time = []
    flux_corr, flux_psf = [], []
    xcom, ycom = [], []

    for i in range(len(star)):

        data0 = eleanor.TargetData(star[i], height=15, width=15, bkg_size=31, do_psf=True, do_pca=False)#
    #add "crowded_field=True" for 3x3 central pixels

        q0 = data0.quality == 0#np.ones(len(data0.time), dtype = bool)

#        plt.plot(data0.time[q0], data0.psf_flux[q0]/np.nanmedian(data0.psf_flux[q0]) + 0.00, 'k.-')
        
        time_eleanor = np.append(time_eleanor, data0.time[q0])
        flux_eleanor_corr = np.append(flux_eleanor_corr, data0.corr_flux[q0]/np.nanmedian(data0.corr_flux[q0]))
        flux_eleanor_psf = np.append(flux_eleanor_psf, data0.psf_flux[q0]/np.nanmedian(data0.psf_flux[q0]))
        xcom, ycom = np.append(xcom, data0.x_com), np.append(ycom, data0.y_com)

#    plt.ylabel('Normalized Flux', fontsize=24)
#    plt.xlabel('Time', fontsize=24)
#    plt.yticks(fontsize=18)
#    plt.xticks(fontsize=18)
#    plt.ylim(0.90, 1.03)

#    plt.show()
    return time_eleanor, flux_eleanor_corr, flux_eleanor_psf, xcom, ycom

In [ ]:
time_eleanor, flux_eleanor_corr, flux_eleanor_psf, x_eleanor_com, y_eleanor_com = get_eleanor_data(tic_, sectors_LC)

In [ ]:
def get_2min_data(tic, sectors_, detrending_, save_file):
    
    tt, ff, ee, qq = [], [], [], []

    for ii in sectors_:        
        try:
            dvt, hdr = ar.getDvt(tic, ii, ext=planetNum, header=True)
            flux = 1. + dvt['LC_DETREND']
            flux_err = dvt['LC_INIT_ERR']
            quality_flags = np.ones(len(flux))
            print('LC_DETREND')
        except:
            dvt, hdr = ar.getLightcurve(tic, ii, ext=planetNum, header=True)
            flux = dvt[detrending_]/np.nanmean(dvt[detrending_])
            flux_err = dvt['SAP_FLUX_ERR']#/np.nanmean(dvt['SAP_FLUX'])
            quality_flags = dvt['QUALITY']
            print(detrending_)
            
        time = dvt['TIME']
                
        tt = np.hstack((tt, time))
        ff = np.hstack((ff, flux))
        ee = np.hstack((ee, flux_err))
        qq = np.hstack((qq, quality_flags))

#flags = np.isnan(ff)

    if save_file == 'y':

        with open(str(tic) + '_SC_S'+str(np.array(sectors_)[0]), mode = 'w') as txt:
            txt_w = csv.writer(txt, delimiter = ' ')
    
#    for time, flux, err in zip(tt[~flags], ff[~flags], ee[~flags], qq[~flags]):
            for time, flux, err, qual in zip(tt, ff, ee, qq):
                txt_w.writerow([time, flux, err, qual])
    return tt, ff, ee, qq

In [ ]:
save_file = 'n'
time_SC, flux_SC, flux_err_SC, quality_SC = get_2min_data(tic_, sectors_SC, detrending_, save_file)

In [ ]:
import csv

save_file_ = 'n'

if save_file_ == 'y':
    with open(str(tic_) + '_Eleanor_LC_S1to10', mode = 'w') as txt:
        txt_w = csv.writer(txt, delimiter = ' ')
        for time, flux in zip(time_eleanor, flux_eleanor):
            txt_w.writerow([time, flux])

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(time_SC, flux_SC, 'k.', label = 'SC')
#plt.plot(time_eleanor, flux_eleanor_corr, 'b-', label = 'Corr')
#plt.plot(time_eleanor, flux_eleanor_psf, 'r-', label = 'PSF')
nn_ = 0
plt.legend()
#plt.xlim(tt_r[0]+nn_*5, tt_r[0]+5+nn_*5)
#plt.xlim(1385, 1395)#1465, 1495.)
#plt.ylim(0.7, 1.03)
#plt.ylim(0.98, 1.02)

In [ ]:
flux_SC.shape

In [ ]:
#plt.plot(ycom)

In [ ]:
#import lightkurve as lk
#pixels = lk.search_targetpixelfile("TIC " + str(tic)).download()

In [ ]:
#lcTIC = LightCurve(time_eleanor, flux_eleanor)

#b=lcTIC.remove_outliers(sigma=5).flatten().to_pandas()

#time_detrend_ = np.asarray(b.time)
#flux_detrend_ = np.asarray(b.flux)

#t0 = min(time_detrend_)
#time_detrend_-= min(time_detrend_)

In [ ]:
#for ii in range(len(time_detrend_)):
#    print(t0+time_detrend_[ii], flux_detrend_[ii])

In [ ]:
m = np.ones(len(flux_eleanor_psf), dtype=bool)

#m = (time_detrend_ > 77.) & (time_detrend_ < 106.)
#m = (time_detrend_ > 145.) & (time_detrend_ < 150.)
#m = (time_detrend_ > 195.) & (time_detrend_ < 200.)
#
#m[(time_detrend_ > 67.) & (time_detrend_ < 70.)] = False
#m[(time_detrend_ > 85.) & (time_detrend_ < 88.)] = False
#m[(time_detrend_ > 155.) & (time_detrend_ < 162.)] = False

#m[(time_detrend_ > 27.) & (time_detrend_ < 30.)] = False
#m[(time_detrend_ > 42.) & (time_detrend_ < 45.)] = False
#m[(time_detrend_ > 110.) & (time_detrend_ < 115.)] = False
#m[(time_detrend_ > 165.) & (time_detrend_ < 167.)] = False
#m[(time_detrend_ > 210.) & (time_detrend_ < 212.)] = False
#m[(time_detrend_ > 267.) & (time_detrend_ < 275.)] = False
#m[(time_detrend_ > 242.) & (time_detrend_ < 245.)] = False

time_detrend_ = time_eleanor[m]
flux_detrend_ = flux_eleanor_psf[m]

In [ ]:
fig = plt.subplots(figsize=(20, 5))
plt.plot(time_detrend_, flux_detrend_, 'k.-')
#plt.xlim(200., 250.)

In [ ]:
# DETREND DATA

def detrend_data_(time, flux, do_plot_, n_points_):
    x = time
    y = flux/np.nanmedian(flux)
    mu = np.median(y)
    y = (y / mu - 1)# * 1e3

# Identify outliers
    m = np.ones(len(y), dtype=bool)
    
    for i in range(10):
        y_prime = np.interp(x, x[m], y[m])
        smooth = savgol_filter(y_prime, n_points_, polyorder=3)
        resid = y - smooth
        sigma = np.sqrt(np.mean(resid**2))
        m0 = np.abs(resid) < 3*sigma
        if m.sum() == m0.sum():
            m = m0
            break
        m = m0
    
    m = (resid < 5*sigma) & (resid > -5*sigma)
    m[(x >= 1353) & (x <= 1354)] = False
    m[(x >= 1422) & (x <= 1424)] = False
    m[(x >= 1463) & (x <= 1465)] = False
#
# Shift the data so that it starts at t=0. This tends to make the fit
# better behaved since t0 covaries with period.
#    print(min(x), min(x[m]))
    x_ref = np.min(x[m])# if sector != 3 else np.min(x)
#    x -= x_ref

# Make sure that the data type is consistent
    x_bak_, y_bak_ = x, y
    x = np.ascontiguousarray(x[m], dtype=np.float64)
    y = np.ascontiguousarray(y[m], dtype=np.float64)
    smooth = np.ascontiguousarray(smooth[m], dtype=np.float64)

    x_detrend_ = x
    y_detrend_ = y - smooth
    
#    do_plot_= 'y'
    if do_plot_ == 'y':
        plt.figure(figsize=(20,5))
    
# Plot the data
#        plt.subplot(211)
        plt.plot(x_bak_+0*x_ref, y_bak_, "k", label="raw data")
        plt.plot(x+0*x_ref, smooth, lw = 3., label = 'sav-gol filter')
        plt.plot(x_bak_[~m]+0*x_ref, y_bak_[~m], "xr", label="outliers")
        plt.legend(fontsize=12)
        plt.xlim(x_bak_.min(), x_bak_.max())
        plt.yticks([])
        plt.xlabel("time")
#        plt.ylabel("flux")
#        plt.ylim(-25., 25.)
        
#        plt.subplot(212)
#        plt.plot(x_detrend_, y_detrend_, "k", label="data")
#        plt.legend(fontsize=12)
#        plt.xlim(x_detrend_.min(), x_detrend_.max())
#        plt.xlabel("time")
#        plt.ylabel("flux")
        plt.show()
    
    return x_detrend_, y_detrend_, x_ref, x_bak_, y_bak_, smooth, m

In [ ]:
#RUN BLS
def run_bls_(x, y, tic, do_plot_, min_period, max_period_):
        
    x_ref = min(x)
    x -= min(x)
    
    period_grid = np.exp(np.linspace(np.log(min_period), np.log(max_period_), 50000))
    durations_ = 0.1+0.05*np.linspace(0,3,4)#0.02+0.02*np.linspace(0,11,12)#0.02+0.02*np.linspace(0,11,12)
    bls_results = []
    periods = []
    t0s = []
    depths = []
    durations = []

    # Compute the periodogram for each planet by iteratively masking out
    # transits from the higher signal to noise planets. Here we're assuming
    # that we know that there are exactly two planets.
    for i in range(1):
        bls = BoxLeastSquares(x, y)
        bls_power = bls.power(period_grid, durations_, oversample=20)
        bls_results.append(bls_power)
    
    # Save the highest peak as the planet candidate
        index = np.argmax(bls_power.power)
        periods.append(bls_power.period[index])
        t0s.append(bls_power.transit_time[index])
        depths.append(bls_power.depth[index])
        durations.append(bls_power.duration[index])
        
    # Mask the data points that are in transit for this candidate
        m = np.zeros(len(x), dtype=bool)
        m |= bls.transit_mask(x, periods[-1], 0.5, t0s[-1])
        
    # PLOT RESULTS   

    if do_plot_ == 'y':

        fig, axes = plt.subplots(len(bls_results), 2, figsize=(20, 10))
        
        for i in range(len(bls_results)):
    # Plot the lightcurve
            plt.subplot(2,3,(1,3))
            plt.plot(x+x_ref, y, 'k-')
            plt.title('detrended TESS lightcurve')
#            plt.ylabel('[ppt]')
            plt.xlim((x + x_ref).min(), (x + x_ref).max())
            plt.yticks([])
    # Plot the periodogram 
            plt.subplot(234)
            plt.plot(np.log10(bls_results[i].period), bls_results[i].power, "k")
#            plt.title('BLS periodogram')
            plt.yticks([])
    # Plot the folded transit (folded on the full period)            
            plt.subplot(235)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < p
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
#            plt.ylim(-10.*depths[0],5*depths[0])
            plt.xlabel('Phase [days]')
#
#
    # Plot the folded transit (folded on 1-day)   
            plt.subplot(236)
            p = periods[i]
            x_fold = (x - t0s[i] + 0.5*p) % p - 0.5*p
            m = np.abs(x_fold) < 1.0 
            plt.plot(x_fold[m], y[m], ".k")  
            inds = np.argsort(x_fold[m])
    # Overplot the box           
            plt.plot(x_fold[m][inds], bls.model(x, p, durations[i], t0s[i])[m][inds], 'C1',lw = 2)
    # Overplot the phase binned light curve
            bins = np.linspace(-1., 1., 32)#
            denom, _ = np.histogram(x_fold, bins)
            num, _ = np.histogram(x_fold, bins, weights=y)
            denom[num == 0] = 1.0
#            plt.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="c", lw = 1)
            plt.title('TIC ' + str(tic) + "; P =" + str(np.round(p,2)) + "; to =" + str(round(x_ref+t0s[0],2)))
            plt.xlabel('Phase [days]')
#            plt.ylim(-10.*depths[0],5*depths[0])
        
            plt.show()
#        plt.savefig("/Users/vkostov/Desktop/Ideas_etc/TESS/tesseract/ryan_LCs/pngs/" + str(tic) + ".png")
#        plt.close()

    return bls_power

In [ ]:
flags = np.isfinite(flux_SC)

detrend_ = 'n'

if detrend_ == 'y':
    do_plot_ = 'y'
    n_points_ = 1221
    time_detrend_, flux_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    detrend_data_(time_SC[flags], flux_SC[flags], do_plot_, n_points_)
    time_detrend_ = time_detrend_# + x_ref
else:
    time_detrend_, flux_detrend_ = time_SC[flags], flux_SC[flags]

In [ ]:
fig = plt.subplots(figsize=(20, 5))
plt.plot(time_detrend_, flux_detrend_, 'k.-')
#plt.xlim(1370., 1400)

In [ ]:
min_period = 5.#30.
max_period = 25.#round(0.8*(max(time_detrend) - min(time_detrend)))

do_plot_ = 'y'
bls_results = run_bls_(time_detrend_, flux_detrend_, tic_, do_plot_, min_period, max_period)
dur_ = 24*bls_results.duration[np.argmax(bls_results.power)]
dep_ = bls_results.depth[np.argmax(bls_results.power)]
print(tic_,
      np.round(bls_results.period[np.argmax(bls_results.power)],2),
      np.round(time_detrend_[0] + bls_results.transit_time[np.argmax(bls_results.power)],4),
      "%.2f" % dur_,
      int(np.round(bls_results.depth[np.argmax(bls_results.power)],0)))

In [ ]:
#idx_SC = np.where((time_SC >= 1482.) & (time_SC <= 1487.))
#time_SC[idx_SC]

In [ ]:
fig = plt.subplots(figsize=(20, 3))

grid = plt.GridSpec(2, 3, wspace=0.4, hspace=0.3)

plt.subplot(141)

idx_LC = np.where((time_eleanor >= 1386.8) & (time_eleanor <= 1391.8))
do_plot_ = 'n'
n_points_ = 121
time_detrend_, flux_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    detrend_data_(time_eleanor[idx_LC], flux_eleanor_psf[idx_LC]/np.nanmedian(flux_eleanor_psf[idx_LC]), do_plot_, n_points_)
plt.plot(time_detrend_, 1.+flux_detrend_, 'k-')
plt.axvspan(1391.27-0.2, 1391.27+0.2, alpha=0.25, color='g')
plt.xlim(1386.8, 1391.8)
plt.ylim(0.99,1.01)

plt.subplot(142)
idx_SC = np.where((time_SC >= 1482) & (time_SC <= 1487))
plt.plot(time_SC[idx_SC], flux_SC[idx_SC]/np.nanmedian(flux_SC[idx_SC]), 'k-', alpha = 0.5)
idx_LC = np.where((time_eleanor >= 1482) & (time_eleanor <= 1487))
time_detrend_, flux_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    detrend_data_(time_eleanor[idx_LC], flux_eleanor_psf[idx_LC]/np.nanmedian(flux_eleanor_psf[idx_LC]), do_plot_, n_points_)
plt.plot(time_detrend_, 1.+flux_detrend_, 'k-')
plt.axvspan(1483.8, 1484.02, alpha=0.25, color='g')
plt.yticks([])
plt.xlim(1482, 1487)
plt.ylim(0.99,1.01)

plt.subplot(143)
idx_SC = np.where((time_SC >= 1575) & (time_SC <= 1580))
plt.plot(time_SC[idx_SC], flux_SC[idx_SC]/np.nanmedian(flux_SC[idx_SC]), 'k-', alpha = 0.5)
idx_LC = np.where((time_eleanor >= 1575) & (time_eleanor <= 1580))
time_detrend_, flux_detrend_, x_ref, x_bak_, y_bak_, smooth, m = \
    detrend_data_(time_eleanor[idx_LC], flux_eleanor_psf[idx_LC]/np.nanmedian(flux_eleanor_psf[idx_LC]), do_plot_, n_points_)
plt.plot(time_detrend_, 1.+flux_detrend_, 'k-')
plt.axvspan(1579.05-0.25, 1579.05+0.25, alpha=0.25, color='g')
plt.yticks([])
plt.xlim(1575., 1580)
plt.ylim(0.99,1.01)

plt.subplot(144)
idx_SC = np.where((time_SC >= 1670) & (time_SC <= 1675))
plt.plot(time_SC[idx_SC], flux_SC[idx_SC]/np.nanmedian(flux_SC[idx_SC]), 'k-', alpha = 0.5)
idx_LC = np.where((time_eleanor >= 1670) & (time_eleanor <= 1675))
plt.plot(time_eleanor[idx_LC], flux_eleanor_psf[idx_LC]/np.nanmedian(flux_eleanor_psf[idx_LC]), 'k.-')
plt.yticks([])
plt.xlim(1670, 1675)
plt.ylim(0.99,1.01)

In [ ]:
#do_plot_ = 'y'
#min_period = 1.
#max_period = int((0.8*(max(time_detrend_) - min(time_detrend_))))
#bls_results = run_bls_(time_detrend_, flux_detrend_, tic_, do_plot_, min_period, max_period)

In [ ]:
index = np.argmax(bls_results.power)

per_ = bls_results.period[index]

per_

In [ ]:
nn = 0

#time_detrend_, flux_detrend_ = data0.time[q0], data0.psf_flux[q0]/np.nanmedian(data0.psf_flux[q0])

per_ = 14.60859501
t0 = x_ref#time_detrend_[0]


n_plots_ = int(max(time_detrend_)/per_)
n_plots_ = 25
fig = plt.subplots(figsize=(20, 50))
for ii in range(n_plots_):

    plt.subplot(n_plots_,1,ii+1)
    
    plt.plot(time_detrend_+x_ref, 1.+flux_detrend_, 'k.-')

    plt.xlim(t0+(ii-nn)*per_,t0+(ii+nn+1)*per_)
    plt.ylim(0.995,1.005)
    
#plt.savefig('tmp_.png')

In [ ]:
fig = plt.subplots(figsize=(20, 5))
flags = np.isfinite(flux_detrend_)
tt_, ff_ = time_detrend_[flags], flux_detrend_[flags]
plt.plot(tt_, ff_)

In [ ]:
do_TB_bls_ = 'y'

if do_TB_bls_ == 'y':

    flags = np.isfinite(flux_detrend_)
    tt_, ff_ = time_detrend_[flags], flux_detrend_[flags]

    period_grid = np.exp(np.linspace(np.log(5.), np.log(25), 50000))
    duration = 0.1+0.05*np.linspace(0,3,4)

    n_planets_ = 2
    bls_periods = np.zeros([n_planets_])
    bls_t0s = np.zeros([n_planets_])
    bls_depths = np.zeros([n_planets_])
    bls_durations = np.zeros([n_planets_])

    
    for ii in range(n_planets_):
# planet 1
        if ii == 0:
            bls = BoxLeastSquares(tt_, ff_)
            bls_power = bls.power(period_grid, duration, oversample=20)

            bls_power0 = bls_power

# Save the highest peak as the planet candidate
            index = np.argmax(bls_power.power)
            bls_period = bls_power.period[index]
            bls_t0 = bls_power.transit_time[index]
            bls_depth = bls_power.depth[index]
            bls_duration = bls_power.duration[index]
            transit_mask = bls.transit_mask(tt_, bls_period, 3*bls_duration, bls_t0)

# planet 2
        else:
            bls = BoxLeastSquares(tt_[~transit_mask], ff_[~transit_mask])
            bls_power = bls.power(period_grid, duration, oversample=20)

# Save the highest peak as the planet candidate
            index = np.argmax(bls_power.power)
            bls_period = bls_power.period[index]
            bls_t0 = bls_power.transit_time[index]
            bls_depth = bls_power.depth[index]
            bls_duration = bls_power.duration[index]
            transit_mask = bls.transit_mask(tt_, bls_period, 3*bls_duration, bls_t0)

        bls_periods[ii] = bls_period
        bls_t0s[ii] = bls_t0
        bls_depths[ii] = bls_depth
        bls_durations[ii] = bls_duration

In [ ]:
bls_periods

In [ ]:
if do_TB_bls_ == 'y':

    fig, axes = plt.subplots(3, 1, figsize=(10, 10))

# Plot the periodogram
    ax = axes[0]
    
    for ii in range(n_planets_):
        ax.axvline(np.log10(bls_periods[ii]), color="C1", lw=5, alpha=0.8)
#    ax.axvline(np.log10(bls_periods[1]), color="C1", lw=5, alpha=0.8)
#    ax.axvline(np.log10(bls_periods[2]), color="C1", lw=5, alpha=0.8)
#    ax.axvline(np.log10(bls_periods[3]), color="C1", lw=5, alpha=0.8)
    ax.plot(np.log10(bls_power0.period), bls_power0.power, "k")
#    ax.annotate("periods = {0:.4f}, {1:.4f}, {2:.4f}, {3:.4f} d".format(*bls_periods),
#                (0, 1), xycoords="axes fraction",
#                xytext=(5, -5), textcoords="offset points",
#                va="top", ha="left", fontsize=12)
    ax.set_ylabel("bls power")
    ax.set_yticks([])
    ax.set_xlim(np.log10(period_grid.min()), np.log10(period_grid.max()))
    ax.set_xlabel("log10(period)")

# Plot the folded transits
    for i in range(n_planets_):
        ax = axes[i+1]
        x_fold = (tt_ - bls_t0s[i] + 0.5*bls_periods[i])%bls_periods[i] - 0.5*bls_periods[i]
        m = np.abs(x_fold) < 0.4
        ax.plot(x_fold[m], ff_[m], ".k")
        inds = np.argsort(x_fold[m])
        ax.plot(x_fold[m][inds], bls.model(tt_[m], bls_periods[i], bls_durations[i], bls_t0s[i])[inds])

    # Overplot the phase binned light curve
        bins = np.linspace(-0.11, 0.11, 52)
        denom, _ = np.histogram(x_fold, bins)
        num, _ = np.histogram(x_fold, bins, weights=ff_)
        denom[num == 0] = 1.0
        ax.plot(0.5*(bins[1:] + bins[:-1]), num / denom, color="C1")

        ax.set_xlim(-0.5, 0.5)
        ax.set_ylabel("de-trended flux [ppt]")
        ax.set_xlabel("time since transit");